In [1]:
!pip install -U transformers accelerate bitsandbytes
!pip install sentencepiece
!pip install langdetect
!pip install translate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [2]:
pip install bitsandbytes


In [3]:
pip install accelerate

In [4]:
from transformers import  (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,TextStreamer,pipeline)
import torch
model_name = "Hemanth-thunder/Tamil-Mistral-7B-Instruct-v0.1"
nf4_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_quant_type="nf4",bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_name,device_map='auto',quantization_config=nf4_config,use_cache=False,low_cpu_mem_usage=True )
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
streamer = TextStreamer(tokenizer)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [5]:
prompt_template ="""சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க, வழங்கப்பட்ட வழிகாட்டுதல்களைப் பின்பற்றி, தேவையான தகவலை உள்ளிடவும்.

### Instruction:
{}

### Response:"""
def create_prompt(query,prompt_template=prompt_template):

    bos_token = "<s>"
    eos_token = "</s>"
    if query:
        lang_code = detect(query)
        print(lang_code)
        if lang_code != "ta":
            print("Query is in {} language so translating to 'ta'".format(lang_code))
            query = translator.translate(query)
            print("Translated-->",query)
        prompt_template = prompt_template.format(query)
    else:
        raise "Please input with query"
    prompt = bos_token+prompt_template #eos_token
    return prompt


In [6]:
print(prompt_template)

சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க, வழங்கப்பட்ட வழிகாட்டுதல்களைப் பின்பற்றி, தேவையான தகவலை உள்ளிடவும்.

### Instruction:
{}

### Response:


In [7]:
streamer = TextStreamer(tokenizer)
pipe = pipeline("text-generation" ,model=model, tokenizer=tokenizer ,
                do_sample=True, repetition_penalty=1.15,top_p=0.95,streamer=streamer)

In [8]:
from translate import Translator
translator = Translator(to_lang="ta")
from langdetect import detect
prompt = create_prompt("How to be healthy in life?")
result=pipe(prompt,max_length=512,pad_token_id=tokenizer.eos_token_id)
print("-------")
print("Finished")

en


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க, வழங்கப்பட்ட வழிகாட்டுதல்களைப் பின்பற்றி, தேவையான தகவலை உள்ளிடவும்.

### Instruction:
வாழ்க்கையில் ஆரோக்கியமாக இருப்பது எப்படி?

### Response:
- ஆரோக்கியமான உணவை சாப்பிடுங்கள். - தவறாமல் உடற்பயிற்சி செய்யுங்கள்.</s>
-------
Finished


In [9]:
pipe = pipeline("text-generation" ,model=model, tokenizer=tokenizer ,do_sample=True, repetition_penalty=1.15,top_p=0.95,streamer=streamer)
prompt = create_prompt("எப்படி மகிழ்ச்சியாக இருக்க வேண்டும்?")
result=pipe(prompt,max_length=512,pad_token_id=tokenizer.eos_token_id)

ta
<s>சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க, வழங்கப்பட்ட வழிகாட்டுதல்களைப் பின்பற்றி, தேவையான தகவலை உள்ளிடவும்.

### Instruction:
எப்படி மகிழ்ச்சியாக இருக்க வேண்டும்?

### Response:
மிகவும் நேர்மறையாகவும் மகிழ்ச்சியாகவும் வாழ, ஒருவரின் உள்ளார்ந்த உந்துதலை அடையாளம் கண்டுகொள்வது அவசியம், இதன் மூலம் நீங்கள் அனுபவிக்கும் மற்றும் விரும்பப்படும் அனுபவங்களில் கவனம் செலுத்தலாம், பின்னர் அதில் நேரத்தை செலவிட நேரம் ஒதுக்கலாம். நண்பர்கள், குடும்பத்தினர் அல்லது பிற வெளிப்புற தூண்டுதல்களுக்கு அப்பால் இருந்து மகிழ்ச்சியை உணரும் பழக்கங்கள். ஒரு நபர் மகிழ்விப்பதற்கான மற்றொரு வழி, பொழுதுபோக்கு அல்லது ஆர்வத்தின் மீது ஆர்வம் காட்ட ஒரு புதிய வேலையைப் பெறுவதாகும். இது அவர்கள் தொடர்ந்து கற்றுக்கொள்வது மற்றும் முன்னேற்றத்தை உணருவதன் மூலம் அவர்களின் ஆளுமைக்கு ஏற்றவாறு வடிவமைக்கப்படலாம். இறுதியாக, சீரான உணவு, உடற்பயிற்சி, தியானம் போன்றவற்றை உள்ளடக்கிய உடல் கவனிப்பை ஒருவர் கடைப்பிடிக்க முடியும். அது கடினமாக இருந்தாலும், நல்ல பழக்கவழக்கங்கள் காலப்போக்கில் வாழ்க்கையை எளிதாக்குகின்றன.</s>
